# Paso 1: Obtencion de datos

In [1]:
import GetOldTweets3 as got


def scrapper(initdate, finaldate ,toptweets, maximo, words=[]):
    s1=[]
    lista_final=[]
    for word in words:
        try:
            s1.append(got.manager.TweetCriteria().setQuerySearch(word).setSince(initdate).setUntil(finaldate).setTopTweets(toptweets).setMaxTweets(int(maximo)))
        except: continue
    
    for i in range (len(s1)):
        lista_final.append(got.manager.TweetManager.getTweets(s1[i]))
    return lista_final

In [10]:
#KeyWords for each candidate
res0= ['Espert', '#Despertar', '@jlespert', '@luisrosalesARG','@FrenteDespertar', '#DejenCompetirAEspert']
res1=['macri', 'pichetto', '#JUNTOSPORELCAMBIO', '@MAURICIOMACRI', '@MIGUELPICHETTO', '@JUNTOSCAMBIOAR']
res2=['@cfkargentina', '@alferdez', 'fernandez-fernandez', '@frentedetodos', 'alberto fernandez']
res3=['@RLavagna', '#consensofederal', 'urtubey']
res4=['Nicolas del caño', '#FITunidad', '@FTE_izquierda', '@NicolasdelCano', '#YoVoteFITUnidad', '@RominaDelPla']
res5=['@juanjomalvinas', '#FrenteNOS']

In [11]:
resultado0=scrapper("2019-10-25", "2019-10-27",False, 3500, res0)#Espert
resultado1=scrapper("2019-10-25", "2019-10-27",False, 3500, res1)#Macri
resultado2=scrapper("2019-10-25", "2019-10-27",False, 3500, res2)#CFK
resultado3=scrapper("2019-10-25", "2019-10-27",False, 3500, res3)#Lavagna
resultado4=scrapper("2019-10-25", "2019-10-27",False, 3500, res4)#Delcaño
resultado5=scrapper("2019-10-25", "2019-10-27",False, 3500, res5)#malvinas

In [12]:
len(resultado0)

6

In [13]:
def extractor(tweets):
    l1,l2,l3,l4,l5,l6=[],[],[],[],[],[]

    for i in range(len(tweets)):
        for tweet in tweets[i]:
            l1.append(tweet.text)
            l2.append(tweet.date)
            l3.append(tweet.username)
            l4.append(tweet.hashtags)
            l5.append(tweet.favorites)
            l6.append(tweet.retweets)
        
    final=pd.DataFrame(list(zip(l1, l2,l3,l4,l5,l6)), 
                       columns =['Tweets', 'Date','User', 'hashtags', 'Favs','RT'])

    
    return final

In [14]:
import pandas as pd

df0=extractor(resultado0)
df1=extractor(resultado1)
df2=extractor(resultado2)
df3=extractor(resultado3)
df4=extractor(resultado4)
df5=extractor(resultado5)

In [15]:
df0.head()

,Tweets,Date,User,hashtags,Favs,RT
0,#VigiliaPorLaLibertad Mañana votan un millón d...,2019-10-26 23:59:52+00:00,GFrondizi,#VigiliaPorLaLibertad,564,170
1,"Pobrecito, le pegaron un pijazo en la nuca y l...",2019-10-26 23:59:51+00:00,cristian_dureOK,,0,0
2,Eso condice con su condicion de seguidora de e...,2019-10-26 23:59:49+00:00,marineroderadio,,2,0
3,Tampoco olvidar el episodio del tiroteo a la c...,2019-10-26 23:59:19+00:00,Maldana__,,2,4
4,Que se lave la jeta este que se cree? Es un de...,2019-10-26 23:59:13+00:00,mariel48465691,,0,0


In [16]:
print(df0.shape)
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(7127, 6)
(13633, 6)
(14794, 6)
(853, 6)
(1306, 6)
(1097, 6)


# Paso2: Eliminando menciones conjuntas

In [52]:
men_FF=['alberto','cfk','cristina','fernandez','fernández','massa','ff','alferdez'] #formas de nombrar a Alberto o a su partido
men_MM=['macri','mm','mauricio','pichetto','vidal'] #formas de nombrar a Mauricio Macri o a su partido
men_FIT=['caño','nico','nicolas','nicolás'] #formas de nombrar a Nicolas del Caño  o a su partido
men_espert=['espert','josé','luis','profesor','rosales'] #formas de nombrar a Espert o a su partido
men_lavagna=['lavagna','roberto','urtubey'] #formas de nombrar a Lavagna o a su partido
men_cent=['gomez','centurion','centurión'] #formas de nombrar a Gomez Centurión o a su partido

In [53]:
others_M= men_FF +  men_FIT + men_espert + men_lavagna + men_cent # mención de otros candidatos en el dataset de Macri
others_FF= men_MM +  men_FIT + men_espert + men_lavagna + men_cent # mención de otros candidatos en el dataset de Alberto Fernandez
others_LAVG= men_FF +  men_FIT + men_espert + men_MM + men_cent # mención de otros candidatos en el dataset de Lavagna
others_FIT= men_FF +  men_MM + men_espert + men_lavagna + men_cent # mención de otros candidatos en el dataset de Nicolás del Caño
others_ESP= men_FF +  men_FIT + men_MM + men_lavagna + men_cent # mención de otros candidatos en el dataset de Espert
others_CENT= men_FF +  men_FIT + men_MM + men_lavagna + men_espert # mención de otros candidatos en el dataset de Gomez Centurión

In [54]:
import re
from nltk import word_tokenize 


In [55]:
from nltk import punkt

In [56]:
def eliminador(tweet,nombres):
    temp=str(tweet)
    
    temp= re.sub(r'\W+', ' ',temp) #elimino simbolos, ya que aplicaremos esta función sobre los tweets sin procesar
                                   # mas abajo se explica el por qué de esto.
                                   
    tokens=word_tokenize(temp) #tokenizamos
    for word in tokens:
        for nombre in nombres:
            x = re.findall(nombre, word.lower()) 
            if x :
                empty=''
                return empty # si encuentra la mención un candidato ajeno al partido, se elimina el tweet
            else:
                continue # continuamos hasta encontrar una coincidencia
    return tweet #si no se encuentra una mención de otro candidato, se retorna el tweet original

In [57]:
dfa=df0.copy()
dfb=df1.copy()
dfc=df2.copy()
dfd=df3.copy()
dfe=df4.copy()
dff=df5.copy()

In [58]:
dfa.head()

,Tweets,Date,User,hashtags,Favs,RT
0,#VigiliaPorLaLibertad Mañana votan un millón d...,2019-10-26 23:59:52+00:00,GFrondizi,#VigiliaPorLaLibertad,563,170
1,"Pobrecito, le pegaron un pijazo en la nuca y l...",2019-10-26 23:59:51+00:00,cristian_dureOK,,0,0
2,Eso condice con su condicion de seguidora de e...,2019-10-26 23:59:49+00:00,marineroderadio,,2,0
3,Tampoco olvidar el episodio del tiroteo a la c...,2019-10-26 23:59:19+00:00,Maldana__,,2,4
4,Que se lave la jeta este que se cree? Es un de...,2019-10-26 23:59:13+00:00,mariel48465691,,0,0


In [59]:
dfa['Tweets']=dfa["Tweets"].map(lambda x : eliminador(x,others_ESP))  #OJO, no aplicarlo sobre columna corregido

In [60]:
dfa.head(50)

,Tweets,Date,User,hashtags,Favs,RT
0,,2019-10-26 23:59:52+00:00,GFrondizi,#VigiliaPorLaLibertad,563,170
1,"Pobrecito, le pegaron un pijazo en la nuca y l...",2019-10-26 23:59:51+00:00,cristian_dureOK,,0,0
2,Eso condice con su condicion de seguidora de e...,2019-10-26 23:59:49+00:00,marineroderadio,,2,0
3,Tampoco olvidar el episodio del tiroteo a la c...,2019-10-26 23:59:19+00:00,Maldana__,,2,4
4,Que se lave la jeta este que se cree? Es un de...,2019-10-26 23:59:13+00:00,mariel48465691,,0,0
5,Hacela peor y anda por Espert ahre,2019-10-26 23:58:56+00:00,Lauchus_,,0,0
6,,2019-10-26 23:58:47+00:00,DanielEcidi,,0,0
7,Grande Profe!!,2019-10-26 23:58:44+00:00,fedebraconi2,,0,0
8,"Grande Espert te vote, te lo dije en aeroparqu...",2019-10-26 23:58:31+00:00,pucciarelli04,,0,0
9,La tuya Giacomini?,2019-10-26 23:57:26+00:00,Gilmour_Floyd1,,0,0


In [61]:
dfa[dfa.Tweets==''].count()

Tweets      1773
Date        1773
User        1773
hashtags    1773
Favs        1773
RT          1773
dtype: int64

In [62]:
dfa=dfa[dfa.Tweets!='']

In [63]:
dfa.shape

(5344, 6)

In [64]:
dfb['Tweets']=dfb["Tweets"].map(lambda x : eliminador(x,others_M))

In [65]:
dfb.head(25)

,Tweets,Date,User,hashtags,Favs,RT
0,,2019-10-26 23:59:58+00:00,joakoaldosivi,,2,0
1,Ni el mismísimo Macri es macrista https://twit...,2019-10-26 23:59:56+00:00,Guillecasanova1,,3,0
2,,2019-10-26 23:59:52+00:00,GFrondizi,#VigiliaPorLaLibertad,563,170
3,Con Macri también hubo medios de transporte a ...,2019-10-26 23:59:50+00:00,GonzarezDC3,,0,0
4,Mi estimada Milita tu cerebrito no da para más...,2019-10-26 23:59:49+00:00,TrigosoAriel,,0,0
5,"Pero resulta que esos años de Macri, Lenín y P...",2019-10-26 23:59:48+00:00,malgraviahhh,,0,2
6,,2019-10-26 23:59:48+00:00,luchorojinegro,,2,1
7,"Pois é, mais Macri não era a solução? O proble...",2019-10-26 23:59:45+00:00,allex_melo,,1,0
8,,2019-10-26 23:59:45+00:00,nerpola_,,0,0
9,Macri macho.,2019-10-26 23:59:44+00:00,EstebanB41,,1,0


In [66]:
dfb=dfb[dfb.Tweets!='']
dfb.shape

(11798, 6)

In [67]:
dfc['Tweets']=dfc["Tweets"].map(lambda x : eliminador(x,others_FF))

In [68]:
dfc.head(20)

,Tweets,Date,User,hashtags,Favs,RT
0,Que no te queres ir a un país que tiene como p...,2019-10-26 23:59:55+00:00,eltuca33,,0,0
1,,2019-10-26 23:59:51+00:00,YoApoyoAChile,,0,0
2,"El mismo narc, digo Kicillof* lo dijo, dejaron...",2019-10-26 23:59:49+00:00,elpicante_,,1,0
3,https://m.facebook.com/story.php?story_fbid=32...,2019-10-26 23:57:45+00:00,pepesiracusa,,0,0
4,Boleta completa y atroden @alferdez @CFKArgent...,2019-10-26 23:57:15+00:00,Roscampitelli,,5,0
5,"Justamente eso es lo que dije siempre ""de las ...",2019-10-26 23:57:04+00:00,LuisReschini,,33,3
6,Andate vos hijo de puta,2019-10-26 23:56:42+00:00,Taliatahiel10,,0,0
7,@Martincho78 boa sorte!!!!!,2019-10-26 23:56:05+00:00,Nina_N___,,1,0
8,,2019-10-26 23:56:02+00:00,aleq45,,0,0
9,Chupala orco,2019-10-26 23:55:56+00:00,spartaco79ing,,0,1


In [69]:
dfc=dfc[dfc.Tweets!='']
dfc.shape

(12651, 6)

In [70]:
dfd['Tweets']=dfd["Tweets"].map(lambda x : eliminador(x,others_LAVG))

In [71]:
dfd=dfd[dfd.Tweets!='']
dfd.shape

(635, 6)

In [72]:
dfe['Tweets']=dfe["Tweets"].map(lambda x : eliminador(x,others_FIT))
dfe=dfe[dfe.Tweets!='']
dfe.shape

(1101, 6)

In [73]:
dff['Tweets']=dff["Tweets"].map(lambda x : eliminador(x,others_CENT))
dff=dff[dff.Tweets!='']
dff.shape

(766, 6)

# Paso 3: Creacion del corpus de entrenamiento

 Ante la necesidad de adaptar el algoritmo a las expresiones y modismos que utilizan los argentinos al momento de hablar sobre política, se decidió armar un corpus propio de entrenamiento y no hacer uso de corpus lingüísticos internacionales, como los que
provee la sociedad española de Natural Language Processing (corpus TASS).  
(De todos modos, para los interesados se deja el link para descargarlo : [corpus TASS](http://www.sepln.org/workshops/tass/tass_data/download.php?auth=SthaMsBsw4leVvKe1r9))  
Para armar este corpus político se clasificó manualmente cerca de 3 mil tweets del total de tweets scrapeados, con las leyendas positivo, negativo y ambiguo, siempre buscando que las 3 clases se encuentren balanceadas (mil tweets clasificados por
cada sentimiento).  

In [74]:
pwd

'/home/alex/Desktop/GitHub_repo/Sentiment_Analysis_Politica-Argentina'

In [75]:
ls

 datasets/                          PASO.ipynb     'Scrapping Sofmanarg.ipynb'
 How_Not_to_Predict_Elections.pdf   PASO-v2.ipynb   Untitled1.ipynb
 imagenes/                          README.md


In [135]:
test_ds=pd.read_excel('datasets/train_dataset_FINAL.xlsx')
test_ds.head()

,Tweets,polaridad
0,"*678, DDD y Barone. *Urtubey y el premio al má...",AMBIGUO
1,*conversación con mi abuelo* -Te tengo que pas...,NEGATIVO
2,*Incomodidad,AMBIGUO
3,"""boletas nefastas"" *se levantan todos* hay uno...",NEGATIVO
4,..atreves hacer comparaciones sin matizes entr...,NEGATIVO


In [136]:
test_ds.shape

(14026, 2)

# Paso 4: Procesamiento del conjunto de entrenamiento

- Se convierte todo el texto a mínuscula
- Se eliminan todo tipo de caracteres no alfabéticos
- Se eliminan todo tipo de URL's 
- Se eliminan nombres de usuario, emojis, etc.
- Se detecta todo tipo de insulto o palabra despectiva y se lo transforma a la etiqueta 'toxicword', con el objetivo de unificar
- Se detecta la mención de algun candidato ya sea por nombre, apellido o apodo y se lo transforma a la etiqueta "somecandidate".
- Se eliminan 'stopwords' (palabras que se repiten con mucha frecuencia pero que no aportan demasiado valor sintáctico, como por ej: de, por, con, ....)
- Se unifican las risas : 'ajjajajajajaj' ----> 'jaja', 'jojojo'----> jaja
- Se lleva a cabo un spellchecking, es decir, se corrigen las palabras mal escritas debido a modismos en redes sociales.  
Ej: 'xq'--> 'porque' , 'q' ---> 'que', etc.
- Se eliminan frases que hagan referencia a cosas dichas por otras personas.  
Por ej: 'El candidato dijo que "tendremos un futuro mejor" ---> 'El candidato dijo que'


In [78]:
import pandas as pd
import numpy as np
import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
stopwords_spanish=stopwords.words('spanish')

In [79]:
stopwords_spanish

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [80]:
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
emoticons = emoticons_happy.union(emoticons_sad)

In [115]:
def text_cleaner(tweet):
        # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'pic.twitter\S+','some_pic',tweet)
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
        # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
      # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_spanish and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            #stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(word)
 
    return ' '.join(tweets_clean)

In [116]:
text_cleaner('EL 11 DE AGOSTO, HAY ALTERNATIVA | #SanFernando es una ciudad que quiere y puede avanzar. Desde @CFederalPBA estamos convencidos de que con equipo, propuestas concretas y los valores que faltan, podemos vivir mejor. @RLavagna @BuccaBali Lista 137 Apic.twitter.com/OJWmiO1oqo')

'11 agosto alternativa sanfernando ciudad quiere puede avanzar at_user convencidos equipo propuestas concretas valores faltan podemos vivir mejor at_user at_user lista 137 asome_pic'

In [117]:
text_cleaner('#11CádizCF| David Gil Iza Carcelen Fali Cala Espino Garrido Bodiger Álex Fernández Salvi Sánchez Javi Navarro Caye Quintana #CádizMálaga #MCFLive')

'11cádizcf david gil iza carcelen fali cala espino garrido bodiger álex fernández salvi sánchez javi navarro caye quintana cádizmálaga mcflive'

In [118]:
text_cleaner('12 años no les alcanzaron para lo IMPORTANTE? Manga de chorros, corruptos!!!! Cómo todo peroncho rufianes y mercenarios!!!!!')

'12 años alcanzaron importante manga chorros corruptos cómo peroncho rufianes mercenarios'

In [119]:
test_ds.head()

,Tweets,polaridad
0,"*678, DDD y Barone. *Urtubey y el premio al má...",AMBIGUO
1,*conversación con mi abuelo* -Te tengo que pas...,NEGATIVO
2,*Incomodidad,AMBIGUO
3,"""boletas nefastas"" *se levantan todos* hay uno...",NEGATIVO
4,..atreves hacer comparaciones sin matizes entr...,NEGATIVO


In [137]:
test_ds['tw_clean']=test_ds.Tweets.apply(lambda x: text_cleaner(x))
test_ds.head(10)

,Tweets,polaridad,tw_clean
0,"*678, DDD y Barone. *Urtubey y el premio al má...",AMBIGUO,678 ddd barone urtubey premio impresentable pa...
1,*conversación con mi abuelo* -Te tengo que pas...,NEGATIVO,conversación abuelo pasar dni averigues voto v...
2,*Incomodidad,AMBIGUO,incomodidad
3,"""boletas nefastas"" *se levantan todos* hay uno...",NEGATIVO,boletas nefastas levantan solo levantó profeso...
4,..atreves hacer comparaciones sin matizes entr...,NEGATIVO,.. atreves hacer comparaciones matizes kirchne...
5,Jajajaja...Naaa ..no votó Peronchos ..y está P...,NEGATIVO,jajajaja ... naaa .. votó peronchos .. pichett...
6,..son mugre no los queremos kukasssss,NEGATIVO,.. mugre queremos kukasss
7,"""Cuando se vaya Macri.Cdo ésto haya pasado"".Se...",AMBIGUO,vaya macri.cdo ésto pasado replica millones ar...
8,....sería de agradecer que ladrones como Macri...,NEGATIVO,... agradecer ladrones macri cría infiltrada a...
9,....sería de agradecer que ladrones como Macri...,NEGATIVO,... agradecer ladrones macri cría infiltrada a...


In [138]:
test_ds.shape

(14026, 3)

In [143]:
from sklearn.model_selection import cross_validate,train_test_split

X_train, X_test, y_train, y_test= train_test_split(test_ds.tw_clean,test_ds.polaridad,test_size=0.2,random_state=42,shuffle=True)

In [147]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11220,)
(11220,)
(2806,)
(2806,)


In [151]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC

pipeline1 = Pipeline([
                ('tfidf', TfidfVectorizer(sublinear_tf=True, # curva logarítmica
                                          ngram_range=(1,2), # uso de bigramas
                                          norm='l2', # se utlizan vectores con norma unitaria
                                          
                                         )),
                
                 ('clf', OneVsRestClassifier(LinearSVC(C=1,multi_class='ovr',random_state=1,tol=0.01)))
                 
            ])


In [152]:
pipeline1.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=True,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight=None,
                                                         dual=True,

In [154]:
prediction=pipeline1.predict(X_test)

In [156]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

report = classification_report(y_test, prediction, output_dict=True)

print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print('positive: ', report['POSITIVO'])
print('negative: ', report['NEGATIVO'])
print('ambiguo: ', report['AMBIGUO'])

Test accuracy is 0.9372772630078403
positive:  {'precision': 0.8461538461538461, 'recall': 0.5789473684210527, 'f1-score': 0.6875, 'support': 133}
negative:  {'precision': 0.9348552338530067, 'recall': 0.9784382284382285, 'f1-score': 0.9561503416856492, 'support': 1716}
ambiguo:  {'precision': 0.9510337323177367, 'recall': 0.9132706374085684, 'f1-score': 0.9317697228144989, 'support': 957}


# Paso 5: Uso del modelo

In [157]:
dfa["tw_clean"]=dfa["Tweets"].map(lambda x : text_cleaner(x)) #espert
dfb["tw_clean"]=dfb["Tweets"].map(lambda x : text_cleaner(x)) #MM
dfc["tw_clean"]=dfc["Tweets"].map(lambda x : text_cleaner(x)) #CFK
dfd["tw_clean"]=dfd["Tweets"].map(lambda x : text_cleaner(x)) #Lav
dfe["tw_clean"]=dfe["Tweets"].map(lambda x : text_cleaner(x)) #delcaño
dff["tw_clean"]=dff["Tweets"].map(lambda x : text_cleaner(x)) #centurion

In [158]:
dfa['predicted_polarity'] = pipeline1.predict(dfa['tw_clean']) 
dfb['predicted_polarity'] = pipeline1.predict(dfb['tw_clean'])
dfc['predicted_polarity'] = pipeline1.predict(dfc['tw_clean'])
dfd['predicted_polarity'] = pipeline1.predict(dfd['tw_clean'])
dfe['predicted_polarity'] = pipeline1.predict(dfe['tw_clean'])
dff['predicted_polarity'] = pipeline1.predict(dff['tw_clean'])


In [171]:
Esp_pred=dfa.groupby('predicted_polarity').tw_clean.count()
Esp_pred

predicted_polarity
AMBIGUO     1118
NEGATIVO    3989
POSITIVO     237
Name: tw_clean, dtype: int64

In [172]:
MM_pred=dfb.groupby('predicted_polarity').tw_clean.count()
MM_pred

predicted_polarity
AMBIGUO     2376
NEGATIVO    7690
POSITIVO    1732
Name: tw_clean, dtype: int64

In [173]:
CFK_pred=dfc.groupby('predicted_polarity').tw_clean.count()
CFK_pred

predicted_polarity
AMBIGUO     3008
NEGATIVO    8089
POSITIVO    1554
Name: tw_clean, dtype: int64

In [174]:
Cano_pred=dfd.groupby('predicted_polarity').tw_clean.count()
Cano_pred

predicted_polarity
AMBIGUO     186
NEGATIVO    252
POSITIVO    197
Name: tw_clean, dtype: int64

In [175]:
malv_pred=dfe.groupby('predicted_polarity').tw_clean.count()
malv_pred

predicted_polarity
AMBIGUO     228
NEGATIVO    738
POSITIVO    135
Name: tw_clean, dtype: int64

# Vote Share Method

VotShare(c1)=[Pc1+Nc2]/[Pc1+Pc2+Nc1+Nc2]

In [178]:
universe=MM_pred['POSITIVO']+CFK_pred['NEGATIVO']+MM_pred['NEGATIVO']+CFK_pred['POSITIVO']
universe

19065

In [180]:
vot_share_MM=(MM_pred['POSITIVO']+CFK_pred['NEGATIVO'])/universe
vot_share_MM.round(2)

0.52

In [182]:
vot_share_CFK=(MM_pred['NEGATIVO']+CFK_pred['POSITIVO'])/universe
vot_share_CFK.round(2)

0.48

# Cloud of Words